<a href="https://colab.research.google.com/github/jstnbshp/discrepancy_report/blob/main/disc_report_test_ver_2_0_072024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from google.colab import files

# Function to generate random dates
def generate_dates(start_date, end_date, num_dates):
    date_range = pd.date_range(start=start_date, end=end_date)
    random_dates = np.random.choice(date_range, num_dates)
    return random_dates

# Parameters for test data
num_records = 100  # Number of records in each dataset
start_date = '2023-01-01'
end_date = '2023-12-31'

# Generate synthetic pixel data
pixel_data = pd.DataFrame({
    'creative_id': np.random.choice(['creative_1', 'creative_2', 'creative_3', 'creative_4'], num_records),
    'date': generate_dates(start_date, end_date, num_records),
    'impressions': np.random.randint(100, 1000, num_records)
})

# Save to CSV
pixel_data.to_csv('pixel_data.csv', index=False)

# Generate synthetic ad server data with metadata header
metadata_header = ["# This is metadata about the file", "# It includes multiple lines", "# Another metadata line"]
ad_server_data = pd.DataFrame({
    'creative_id_1': np.random.choice(['creative_1', 'creative_2', 'creative_3', 'creative_4'], num_records),
    'creative_id_2': np.random.choice(['creative_A', 'creative_B', 'creative_C', 'creative_D'], num_records),
    'creative_id_3': np.random.choice(['creative_X', 'creative_Y', 'creative_Z'], num_records),
    'date': generate_dates(start_date, end_date, num_records),
    'impressions': np.random.randint(100, 1000, num_records)
})

# Save to CSV with metadata header
with open('ad_server_raw_data.csv', 'w') as f:
    for line in metadata_header:
        f.write(line + '\n')
    ad_server_data.to_csv(f, index=False)

# Upload files to Google Colab
uploaded = files.upload()

# Load the data, skipping the metadata header (assume 3 lines of metadata)
pixel_data = pd.read_csv('pixel_data.csv')
ad_server_data = pd.read_csv('ad_server_raw_data.csv', skiprows=3)

# Ensure consistent datetime formats
pixel_data['date'] = pd.to_datetime(pixel_data['date'], errors='coerce')
ad_server_data['date'] = pd.to_datetime(ad_server_data['date'], errors='coerce')

# Preprocess ad server data
def preprocess_ad_server_data(df):
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    for col in ['creative_id_1', 'creative_id_2', 'creative_id_3']:
        df[col] = df[col].astype(str).str.strip()
    df['impressions'] = df['impressions'].fillna(0)
    return df

ad_server_data = preprocess_ad_server_data(ad_server_data)

# Function to match IDs
def match_ids(ad_df, pixel_df):
    matches = []
    for ad_col in ['creative_id_1', 'creative_id_2', 'creative_id_3']:
        temp_df = pd.merge(ad_df[[ad_col, 'date', 'impressions']], pixel_df, left_on=[ad_col, 'date'], right_on=['creative_id', 'date'], suffixes=('_ad_server', '_pixel'))
        matches.append(temp_df)
    return pd.concat(matches).drop_duplicates()

# Match IDs and merge data
comparison = match_ids(ad_server_data, pixel_data)

# Aggregate total impressions by creative_id for both datasets
ad_server_agg = ad_server_data.groupby(['creative_id_1', 'creative_id_2', 'creative_id_3']).agg({'impressions': 'sum'}).reset_index()
pixel_agg = pixel_data.groupby(['creative_id']).agg({'impressions': 'sum'}).reset_index()

# Merge the two datasets on creative_id
comparison = pd.merge(ad_server_agg, pixel_agg, left_on='creative_id_1', right_on='creative_id', suffixes=('_ad_server', '_pixel'))

# Calculate the difference in impressions and percentage difference
comparison['impression_diff'] = comparison['impressions_ad_server'] - comparison['impressions_pixel']
comparison['percent_diff'] = ((comparison['impression_diff'] / comparison['impressions_pixel']) * 100).round(2).astype(str) + '%'

# Identify discrepancies
discrepancies = comparison[comparison['impression_diff'] != 0]

# Save the results to an Excel file with multiple sheets
with pd.ExcelWriter('discrepancies.xlsx') as writer:
    discrepancies.to_excel(writer, sheet_name='Discrepancies', index=False)
    pixel_data.to_excel(writer, sheet_name='Pixel Data', index=False)
    ad_server_data.to_excel(writer, sheet_name='Ad Server Data', index=False)

# Download the Excel file
files.download('discrepancies.xlsx')

Saving ad_server_raw_data (1).csv to ad_server_raw_data (1).csv
Saving pixel_data (3).csv to pixel_data (3) (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Product Requirements Document (PRD)

### Project Title: Advertising Data Comparison and Discrepancy Reporting

### Introduction
This project aims to develop a system that compares digital advertising impression data from ad servers and pixel data. The primary objective is to identify discrepancies in total impressions for different creative IDs across multiple days, calculate the percentage difference, and generate a detailed report. The system will use Google Colab for initial testing and development and later migrate to a production environment.

### Objectives
1. **Data Ingestion**: Load ad server and pixel data into the system.
2. **Data Preprocessing**: Clean and standardize the ad server data, ignoring metadata headers.
3. **Metadata Integration**: Use a Language Model (LLM) to enrich creative IDs with metadata.
4. **Data Aggregation**: Sum impressions by creative ID.
5. **Comparison and Discrepancy Identification**: Calculate differences and percentage discrepancies between datasets.
6. **Reporting**: Generate a detailed report with discrepancies and save it as an Excel file with multiple sheets.

### Requirements

#### Functional Requirements
1. **Data Ingestion**:
   - The system must be able to upload CSV files for ad server data and pixel data.
   - The system should handle errors in file upload gracefully.
   - The system must be able to skip metadata headers in the ad server data.

2. **Data Preprocessing**:
   - Standardize date formats across datasets.
   - Clean and normalize creative IDs (e.g., remove leading/trailing spaces).
   - Handle missing values by filling them with appropriate defaults (e.g., zero for impressions).

3. **Metadata Integration**:
   - Implement a function to enrich creative IDs with metadata using an LLM or mock function.
   - Metadata should include attributes like advertisement type and category.
   - Create a separate source file to map known terminology and issues. This source file will be used to analyze the metadata in the ad server data and output it in the discrepancy report.

4. **Data Aggregation**:
   - Aggregate data by summing impressions for each creative ID, ignoring the date.

5. **Comparison and Discrepancy Identification**:
   - Merge aggregated datasets on creative ID.
   - Calculate the absolute difference and percentage difference in impressions.
   - Format the percentage difference to two decimal places and append a percent symbol.

6. **Reporting**:
   - Generate a report of discrepancies and save it as an Excel file with separate sheets for discrepancies, pixel data, and ad server data.
   - Ensure the Excel file is formatted for easy viewing in applications like Microsoft Excel.

### Differences Between SQL and Python for This Task

#### SQL
**Advantages**:
1. **Data Handling in Databases**: SQL is inherently designed for querying and managing data stored in relational databases. If your data is already in a database, SQL is the most direct way to query and manipulate it.
2. **Aggregation and Filtering**: SQL excels at operations like aggregation, filtering, and joining tables, which are fundamental to this task.
3. **Performance**: SQL queries are optimized for performance in database engines. Complex joins and aggregations are often faster when executed directly within the database.
4. **Simplicity for Basic Operations**: For straightforward data extraction, transformation, and loading (ETL) tasks, SQL is often simpler and more readable.

**Limitations**:
1. **Complex Data Manipulations**: SQL can become cumbersome and less readable for very complex data manipulations and transformations.
2. **Flexibility**: SQL lacks the flexibility of a full programming language for advanced data processing and integrating with other tools and libraries.

#### Python
**Advantages**:
1. **Complex Data Processing**: Python, with libraries like Pandas and NumPy, is highly suited for complex data transformations and manipulations.
2. **Integration**: Python can easily integrate with various data sources and other tools, making it ideal for end-to-end data workflows.
3. **Data Visualization**: Python offers powerful libraries like Matplotlib, Seaborn, and Plotly for data visualization, which can be useful for analyzing and presenting your findings.
4. **Machine Learning and Advanced Analytics**: If your task involves machine learning or advanced analytics, Python provides extensive libraries and frameworks to support these tasks.

**Limitations**:
1. **Performance**: For very large datasets, Python might be slower compared to SQL executed directly within a database.
2. **Setup and Environment**: Using Python requires a setup that includes the necessary libraries and possibly an integrated development environment (IDE).

### Test Plan

#### Testing in SQL

1. **Data Preparation**:
   - Create tables `pixel_data` and `ad_server_data`.
   - Insert test data into the tables.

2. **Data Preprocessing**:
   - Write SQL queries to clean and preprocess the data, ignoring metadata headers.

3. **Data Aggregation**:
   - Write SQL queries to sum impressions by creative ID.

4. **Comparison and Discrepancy Identification**:
   - Write SQL queries to calculate absolute and percentage differences in impressions.

5. **Reporting**:
   - Write SQL queries to generate the discrepancy report.

#### Testing in Python

1. **Data Preparation**:
   - Load the data files into pandas DataFrames in Google Colab.
   - Handle metadata headers during data loading.

2. **Data Preprocessing**:
   - Use pandas to clean and normalize the data.

3. **Data Aggregation**:
   - Use pandas to sum impressions by creative ID.

4. **Comparison and Discrepancy Identification**:
   - Use pandas to calculate absolute and percentage differences in impressions.

5. **Reporting**:
   - Use pandas and openpyxl to generate the discrepancy report and save it as an Excel file with multiple sheets.

### Milestones

1. **Week 1**:
   - Set up Google Colab environment.
   - Implement data ingestion and preprocessing.

2. **Week 2**:
   - Integrate metadata enrichment function.
   - Implement data aggregation logic.

3. **Week 3**:
   - Develop comparison and discrepancy identification logic.
   - Format percentage differences correctly.

4. **Week 4**:
   - Generate the Excel report with multiple sheets.
   - Conduct testing with SQL and Python implementations.

5. **Week 5**:
   - Review and refine the system based on feedback.
   - Prepare for migration to the production environment.

### Stakeholders
- **Project Manager**: Oversees the project timeline and deliverables.
- **Data Analysts**: Use the system to compare and analyze advertising data.
- **Developers**: Implement the system functionalities.
- **IT Security**: Ensure data security and compliance.
- **End Users**: Individuals who will upload data and review reports.

### Appendix
- **Sample Data Files**: Provide examples of ad server data and pixel data CSV files for testing.
- **Metadata Mapping**: Documentation on the attributes added to creative IDs via the LLM.
- **Technical Specifications**: Detailed documentation on the preprocessing, aggregation, and comparison algorithms used.

### Approval
- **Project Sponsor**: [Name]
- **Date**: [Date]

---

This PRD outlines the scope, requirements, and timeline for developing the advertising data comparison and discrepancy reporting system. Please review and provide feedback or approval to proceed with development.